# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1: Marta Castillo Galán
*   Alumno 2: Unai Marín Etxebarria
*   Alumno 3: Laura Molinos Mayo
*   Alumno 4: Raúl Murillo Gallego






---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

In [10]:
#if IN_COLAB:
  #%pip install gym==0.17.3
  #%pip install git+https://github.com/Kojoley/atari-py.git
  #%pip install keras-rl2==1.0.5
  #%pip install tensorflow==2.8
#else:
  #%pip install gym==0.17.3
  #%pip install git+https://github.com/Kojoley/atari-py.git
  #%pip install pyglet==1.5.0
  #%pip install h5py==3.1.0
  #%pip install Pillow==9.5.0
  #%pip install keras-rl2==1.0.5
  #%pip install Keras==2.2.4
  #%pip install tensorflow==2.5.3
  ##%pip install torch==2.0.1
  #%pip install agents==1.4.0

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [11]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Conv2D
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

#### Configuración base

In [12]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [13]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1. Implementación de la red neuronal

In [14]:

from tensorflow.keras.layers import  Dropout

def build_model(window_length, input_shape, nb_actions):
    model = Sequential()
    model.add(Permute((2, 3, 1), input_shape=(window_length,) + input_shape))  # (C, H, W) → (H, W, C)

    # Convolucional 1
    model.add(Conv2D(32, (8, 8), strides=(4, 4), padding='same'))
    model.add(Activation('relu'))
    
    # Convolucional 2
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(Activation('relu'))
    
    # Convolucional 3 (ligero cambio en tamaño del kernel)
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same'))
    model.add(Activation('relu'))

    model.add(Flatten())

    # Capa densa + regularización
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))  # 25% dropout

    # Salida
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    
    return model


In [15]:
from rl.policy import Policy
import numpy as np
import random

class FireBiasPolicy(Policy):
    def __init__(self, fire_action_idx, nb_steps_bias=10000):
        self.fire_action = fire_action_idx
        self.step = 0
        self.nb_steps_bias = nb_steps_bias

    def select_action(self, q_values):
        self.step += 1

        if self.step < self.nb_steps_bias:
            # Alta probabilidad de disparar (por ejemplo 30%)
            if random.random() < 0.3:
                return self.fire_action

        # Si no, acción greedy
        return np.argmax(q_values)


2. Implementación de la solución DQN

En este intento, tratamos de combinar la capacidad de un modelo anterior para esquivar las balas contrarias con una estrategia predisopuesta mayoritariamente a que el agente dispare. Haremos fine tuning a los pesos del modelo "dqn_spaceinvaders_weights_aprende_a_esquivar". El objetivo es por lo tanto que el agente aprenda lo mejor de ambas estrategias.

In [21]:
# Preparamos memoria y estrategia
memory = SequentialMemory(limit=100000, window_length=WINDOW_LENGTH)
#policy = EpsGreedyQPolicy()  # Exploración simple
policy = FireBiasPolicy(fire_action_idx=1, nb_steps_bias=20000)
# Creamos el modelo
model = build_model(WINDOW_LENGTH, INPUT_SHAPE, nb_actions)
# 2. Luego cargas los pesos
model.load_weights('dqn_spaceinvaders_weights_aprende_a_esquivar.h5f')
print(model.summary())

# Creamos el procesador
processor = AtariProcessor()
print(env.unwrapped.get_action_meanings())

#Creamos el agente
dqn = DQNAgent(model=model,
               nb_actions=nb_actions,
               policy=policy,
               memory=memory,
               processor=processor,
               nb_steps_warmup=10000,
               gamma=0.99,
               target_model_update=10000,
               train_interval=4,
               delta_clip=1.0)

dqn.compile(Adam(learning_rate=1e-4), metrics=['mae'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_2 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 32)        8224      
_________________________________________________________________
activation_5 (Activation)    (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        32832     
_________________________________________________________________
activation_6 (Activation)    (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 512)       295424    
_________________________________________________________________
activation_7 (Activation)    (None, 11, 11, 512)      

In [17]:
# Entrenamiento inicial
# (0.0.0.2) Pasamos de 5000 a 20.000 pasos
dqn.fit(env, nb_steps=20000, visualize=False, verbose=2)

Training for 20000 steps ...


C:\Users\Unai\anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


   531/20000: episode: 1, duration: 14.939s, episode steps: 531, steps per second:  36, episode reward: 11.000, mean reward:  0.021 [ 0.000,  1.000], mean action: 1.908 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1152/20000: episode: 2, duration: 13.169s, episode steps: 621, steps per second:  47, episode reward: 14.000, mean reward:  0.023 [ 0.000,  1.000], mean action: 1.646 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1734/20000: episode: 3, duration: 11.412s, episode steps: 582, steps per second:  51, episode reward: 12.000, mean reward:  0.021 [ 0.000,  1.000], mean action: 1.667 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2954/20000: episode: 4, duration: 23.715s, episode steps: 1220, steps per second:  51, episode reward: 17.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 1.645 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  3544/20000: episode: 5, duration: 12.178s, episode steps: 590, steps per second:  48, episode reward: 10.000, mean reward:  0.017 [ 0

C:\Users\Unai\anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 10225/20000: episode: 13, duration: 41.085s, episode steps: 880, steps per second:  21, episode reward: 20.000, mean reward:  0.023 [ 0.000,  1.000], mean action: 1.719 [0.000, 5.000],  loss: 0.007922, mae: 0.029687, mean_q: 0.060974
 11067/20000: episode: 14, duration: 96.972s, episode steps: 842, steps per second:   9, episode reward: 20.000, mean reward:  0.024 [ 0.000,  1.000], mean action: 1.467 [0.000, 5.000],  loss: 0.007545, mae: 0.035066, mean_q: 0.062506
 12175/20000: episode: 15, duration: 128.972s, episode steps: 1108, steps per second:   9, episode reward: 22.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 1.609 [0.000, 5.000],  loss: 0.008086, mae: 0.038569, mean_q: 0.065566
 13300/20000: episode: 16, duration: 129.316s, episode steps: 1125, steps per second:   9, episode reward: 20.000, mean reward:  0.018 [ 0.000,  1.000], mean action: 1.467 [0.000, 5.000],  loss: 0.009048, mae: 0.041562, mean_q: 0.070368
 13663/20000: episode: 17, duration: 42.607s, episode st

In [ ]:
#Evaluación en modo test
scores = dqn.test(env, nb_episodes=10, visualize=False)
print("Media de recompensa:", np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...


C:\Users\Unai\anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Episode 1: reward: 17.000, steps: 751
Episode 2: reward: 14.000, steps: 1162
Episode 3: reward: 19.000, steps: 1263
Episode 4: reward: 16.000, steps: 797
Episode 5: reward: 10.000, steps: 616
Episode 6: reward: 17.000, steps: 1263
Episode 7: reward: 12.000, steps: 1127


In [19]:
#Guardado de pesos
STRESRSRSRdqn.save_weights('dqn_spaceinvaders_weights.h5f', overwrite=True)

In [20]:
## Testing part to calculate the mean reward
#weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
#dqn.load_weights(weights_filename)
#dqn.test(env, nb_episodes=10, visualize=False)

3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---